#### Setting environments

In [1]:
import torch, os, gc
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,\
f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

In [2]:
# Set options
embed_ver = ["clstm", "esm2", "bert", "t5"]
test_ver = ["C018", "C039", "C035", "D011", "C048"]

# model options
data_path = "../data/test_exam/"
model_path = f"../models/cls/"
result_path = f"../results/"
os.makedirs(result_path + "prd-indiv/", exist_ok=True)

col_str = ['file_id', 'organism', 'locus_tag', 'ess']

batch_size = 256

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Define function to record perfomance result
def record_perform(emb_ver, file_id, organ, y_real, y_conf, y_prd):    
    if file_id != "O046":
        auc_roc = [roc_auc_score(y_real, y_conf)]
        auc_pr = [average_precision_score(y_real, y_conf)]
    else:
        auc_roc = None
        auc_pr = None
    
    tn, fp, fn, tp = confusion_matrix(y_real, y_prd).ravel()
    
    result = pd.DataFrame({
        "emb": [emb_ver],
        "file": [file_id],
        "organism": [organ],
        "tp": [tp],
        "fp": [fp],
        "tn": [tn],
        "fn": [fn],
        "mcc": [matthews_corrcoef(y_real, y_prd)],
        "acc": [accuracy_score(y_real, y_prd)],
        "f1": [f1_score(y_real, y_prd)],
        "prc": [precision_score(y_real, y_prd)],
        "rec": [recall_score(y_real, y_prd)],
        "npv": [precision_score(1 - y_real, 1 - y_prd)],
        "tnr": [recall_score(1 - y_real, 1 - y_prd)],
        "auc-roc": auc_roc,
        "auc-pr": auc_pr
    })

    return result


In [5]:
# Set model architecture
class Classifier(nn.Module):
    def __init__(self, input_size, num_layers, unit_decrease):
        super(Classifier, self).__init__()
        layers = [nn.BatchNorm1d(input_size), nn.Dropout(0.5)]
        in_dim = input_size
        out_dim = 1024
        for i in range(num_layers):            
            out_dim = max(2, out_dim // unit_decrease)
            layers.append(nn.Linear(in_dim, out_dim))
            self.initialize_weights(layers[-1])
            layers.append(nn.GELU())
            in_dim = out_dim
        layers.append(nn.Linear(out_dim, 1))
        self.cls_block = nn.Sequential(*layers)
        
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        return self.cls_block(x)

#### Evaluate model

In [6]:
df_eval = pd.DataFrame()

for ver in embed_ver:
    print(f"\n>>>> {ver} <<<<")

    total_label = []
    total_conf = []
    total_cls = []
    for ts_ver in test_ver:    
        # load dataset
        data = pd.read_csv(data_path + f"data-emb_{ver}-{ts_ver}_ts.csv")
        display("Raw data:", data)

        #### Preprocess for test dataset ####
        col_num = [col for col in data.columns if col not in col_str]

        # split info.& inputs & labels of the test datasets
        info_ts = data[col_str]
        X_ts = torch.tensor(data[col_num].astype('float32').values)
        y_ts = torch.tensor(data['ess'].astype('float32').values)
        print(f"Splited test dataset({ts_ver}):", X_ts.shape, y_ts.shape)
        
        # generate dataloader by the test datasets
        dataset_ts = TensorDataset(X_ts, y_ts)
        test_loader = DataLoader(dataset_ts, batch_size=batch_size, shuffle=False)

        ## Test model ##    
        # generate model instance
        model = Classifier(
            input_size=X_ts.shape[-1],
            num_layers=3,
            unit_decrease=2
        ).to(device)

        # load model weight
        model.load_state_dict(torch.load(f"{model_path}{ver}-{ts_ver}.pt", map_location=device))
        model.eval()

        ## model evaluations by test dataset ##
        results = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                # prediction
                preds = model(X_batch).view(-1).cpu()
                # gather the predictions
                results.append(preds)
            
        results = torch.cat(results, dim=0)
        
        # convert logits to confidences & classes
        prd_conf = torch.sigmoid(results)
        prd_cls = (prd_conf >= 0.5).int().tolist()

        # save the model prediction result
        df_pred = pd.concat([info_ts, pd.DataFrame({"conf": prd_conf.tolist()})], axis=1)
        df_pred.to_csv(f"{result_path}prd-indiv/{ver}-{ts_ver}.csv", index=False)
        
        # gather the results
        total_label.extend(df_pred['ess'].tolist())
        total_conf.extend(df_pred['conf'].tolist())
        total_cls.extend(prd_cls)
        
        # get evaluation row by testset
        eval_ts = record_perform(
            emb_ver=ver,
            file_id=ts_ver,
            organ=ts_ver,
            y_real=df_pred['ess'].to_numpy(),
            y_conf=df_pred['conf'].to_numpy(),
            y_prd=np.array(prd_cls)
        )
        df_eval = pd.concat([df_eval, eval_ts], ignore_index=True)
        print(f"- Test in {ts_ver} was done.")
    
    del model
    gc.collect()
    torch.cuda.empty_cache()

    # get total mean row
    eval_ts = record_perform(
        emb_ver=ver,
        file_id="total",
        organ="all",
        y_real=np.array(total_label),
        y_conf=np.array(total_conf),
        y_prd=np.array(total_cls)
    )
    df_eval = pd.concat([df_eval, eval_ts], ignore_index=True)
    print(f"- Test in total testset was done.")

# save the model perfomance result
df_eval.to_csv(f"{result_path}eval-indiv_cls-strain.csv", index=False)
display("Model performance:", df_eval)


>>>> clstm <<<<


'Raw data:'

,file_id,organism,locus_tag,ess,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,C018,Escherichia coli K-12 BW25113,BW25113_0001,0,-1.141337,0.000117,0.074418,-0.000446,-0.002198,-0.003431,...,-0.000123,0.004792,0.000471,0.126904,0.006709,0.013690,-0.021874,-0.049524,0.108785,-0.003517
1,C018,Escherichia coli K-12 BW25113,BW25113_0002,0,0.137700,0.000258,0.067561,-0.000478,-0.000678,-0.003556,...,-0.000045,0.001738,0.002048,0.126786,0.007048,0.008776,0.114241,-0.035615,0.105729,0.001858
2,C018,Escherichia coli K-12 BW25113,BW25113_0003,0,0.128075,0.000037,0.062052,-0.004017,-0.002748,-0.000468,...,-0.000051,0.001672,0.002281,0.126932,0.007019,0.007990,0.115545,-0.033727,0.102622,0.002154
3,C018,Escherichia coli K-12 BW25113,BW25113_0004,0,0.487113,0.000235,0.064557,-0.001244,-0.000863,-0.002970,...,-0.000038,0.001291,0.000991,0.127128,0.007119,0.008230,0.117134,-0.032810,0.105667,0.001789
4,C018,Escherichia coli K-12 BW25113,BW25113_0005,0,-0.501022,0.000158,0.070695,-0.000914,-0.001795,-0.003404,...,-0.000019,-0.000853,0.003466,0.126817,0.007367,0.006851,0.123346,-0.026789,0.104581,0.004456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4308,C018,Escherichia coli K-12 BW25113,BW25113_4702,0,-0.824822,0.000154,0.070713,-0.000348,-0.002115,-0.004313,...,-0.000115,0.004844,0.001291,0.126999,0.006747,0.013313,-0.021632,-0.051901,0.107055,-0.004071
4309,C018,Escherichia coli K-12 BW25113,BW25113_4703,0,-1.325105,0.000108,0.077958,-0.000324,-0.002058,-0.002888,...,-0.000106,0.004112,0.002155,0.127117,0.006835,0.011485,0.009256,-0.044031,0.106437,-0.002243
4310,C018,Escherichia coli K-12 BW25113,BW25113_4705,0,-1.137151,0.000133,0.076093,-0.000250,-0.001809,-0.003093,...,-0.000059,0.002871,0.002066,0.127147,0.006998,0.009537,0.041613,-0.044984,0.104478,-0.000265
4311,C018,Escherichia coli K-12 BW25113,BW25113_4706,0,-1.110526,0.000127,0.074501,-0.000332,-0.002103,-0.003567,...,-0.000113,0.004755,0.000375,0.126987,0.006722,0.013741,-0.018794,-0.051945,0.108058,-0.003450


Splited test dataset(C018): torch.Size([4313, 1024]) torch.Size([4313])
- Test in C018 was done.


'Raw data:'

...

'Raw data:'

,file_id,organism,locus_tag,ess,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0001,0,-0.025583,-0.004581,0.078834,-0.007245,-0.017835,0.027827,...,-0.065012,-0.028501,0.053340,-0.077918,0.056029,0.041364,-0.014699,-0.001872,0.004338,0.001955
1,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0002,0,0.018134,0.013939,0.075287,0.070802,-0.001094,0.025817,...,-0.021290,-0.020542,-0.034380,-0.072337,0.068918,0.010234,0.028582,0.009465,0.093650,0.043395
2,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0003,0,0.011903,-0.050535,0.087673,0.071201,-0.009868,0.021778,...,-0.043901,-0.007869,-0.009904,-0.089165,0.022759,-0.034698,0.015764,-0.004712,0.054782,0.000353
3,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0004,1,-0.010374,0.003995,-0.002378,0.038072,-0.007250,0.027236,...,-0.049503,-0.045843,0.005756,-0.022377,-0.010699,0.019500,-0.003762,0.031871,0.039024,-0.007551
4,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0005,0,-0.011068,-0.041055,0.006833,0.067446,0.008203,-0.021138,...,-0.043943,-0.005162,0.013928,-0.136281,0.067658,-0.013946,-0.021438,0.000856,0.044038,-0.004827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4820,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548234,0,0.031741,-0.001645,0.019326,-0.011843,-0.052117,0.047029,...,-0.054037,0.004018,-0.020355,-0.100695,-0.033680,-0.015619,-0.041882,-0.075407,0.059378,-0.028652
4821,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548235,0,0.011300,-0.025902,-0.010313,-0.032185,-0.015765,0.040253,...,-0.043232,-0.013576,-0.046755,-0.065286,0.012180,0.054107,-0.003031,-0.025053,-0.015699,0.035863
4822,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548236,0,-0.016179,0.049258,-0.010011,0.008940,-0.032092,0.033933,...,-0.017677,-0.053039,-0.022061,-0.076991,0.006883,0.069524,-0.009119,0.030341,0.042864,-0.057160
4823,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548237,0,-0.013817,0.125102,0.030937,0.018625,-0.055700,-0.024432,...,0.065290,-0.014176,-0.032757,-0.104375,-0.026381,0.102121,0.083062,0.130213,0.057213,-0.048246


Splited test dataset(C048): torch.Size([4825, 1024]) torch.Size([4825])
- Test in C048 was done.
- Test in total testset was done.


'Model performance:'

,emb,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm,C018,C018,233,229,3786,65,0.594411,0.931834,0.613158,0.504329,0.781879,0.983121,0.942964,0.910965,0.586572
1,clstm,C039,C039,206,356,4863,145,0.418455,0.910054,0.451260,0.366548,0.586895,0.971046,0.931788,0.866708,0.489602
2,clstm,C035,C035,241,282,3124,239,0.404242,0.865929,0.480558,0.460803,0.502083,0.928933,0.917205,0.814435,0.513969
3,clstm,D011,D011,128,375,2234,155,0.241870,0.816736,0.325700,0.254473,0.452297,0.935119,0.856267,0.725506,0.275557
4,clstm,C048,C048,110,587,3913,215,0.148307,0.833782,0.215264,0.157819,0.338462,0.947917,0.869556,0.656366,0.172057
5,clstm,total,all,918,1829,17920,819,0.355830,0.876757,0.409456,0.334183,0.528497,0.956294,0.907388,0.790771,0.362973
6,esm2,C018,C018,275,492,3523,23,0.530793,0.880594,0.516432,0.358540,0.922819,0.993514,0.877460,0.944574,0.719505
7,esm2,C039,C039,331,430,4789,20,0.608896,0.919210,0.595324,0.434954,0.943020,0.995841,0.917609,0.976373,0.813541
8,esm2,C035,C035,416,445,2961,64,0.583130,0.869017,0.620433,0.483159,0.866667,0.978843,0.869348,0.939435,0.766418
9,esm2,D011,D011,225,375,2234,58,0.477245,0.850277,0.509626,0.375000,0.795053,0.974695,0.856267,0.894539,0.686368
